In [5]:
!pip install opencv-python

  Using cached opencv_python-4.5.1.48-cp38-cp38-macosx_10_13_x86_64.whl (40.3 MB)


In [7]:
!pip install imutils

Processing /Users/francois/Library/Caches/pip/wheels/59/1b/52/0dea905f8278d5514dc4d0be5e251967f8681670cadd3dca89/imutils-0.5.4-py3-none-any.whl


In [8]:
#Avant d'exécuter ce fichier, assurez-vous que vous avez le dossier Pretrained model et classes
#vous pouvez tout obtenir via mon lien github que je mentionnerai ci-dessous
# Cela dépendra des performances du processeur si les performances de votre processeur sont bonnes,
# la vidéo sera traitée rapidement # je ne l'ai pas bonne performance cpu à ce pourquoi son traitement assez faible
import cv2
# openCV est utilisé pour toutes sortes d'analyses d'images et de vidéos,
# telles que la reconnaissance et la détection faciales, la lecture de plaques d'immatriculation,
# l'édition de photos, la vision robotique avancée, la reconnaissance optique de caractères, et bien plus encore.
import datetime
# Le module datetime permet de manipuler les dates et les heures.
import imutils
# imutis compose une série de fonctions pratiques pour rendre les fonctions de traitement d'image de base telles que
# la traduction,la rotation, le redimensionnement, le squelette, l'affichage des images Matplotlib, le tri des contours,
# la détection des bords et bien plus encore avec OpenCV et Python .

import numpy as np
# NumPy est une extension du langage de programmation Python,
# destinée à manipuler des matrices ou tableaux multidimensionnels ainsi que
# des fonctions mathématiques opérant sur ces tableaux
from centroidtracker import CentroidTracker
from itertools import combinations
#Ce module implémente de nombreuses briques d'itérateurs rapides et efficaces pour boucler efficacement
import math
# Ce module math permet d'accéder aux fonctions mathématiques définies par le standard C

In [9]:
#Le modèle et les fichiers prototypes sont ici
protopath = "MobileNetSSD_deploy.prototxt"
modelpath = "MobileNetSSD_deploy.caffemodel"

# modelConfiguration = "C:\\Users\\Dass\Desktop\\test\\Computer-Vision\\Social_distancing\\yolov3.cfg"
# modelWeight = "C:\\Users\\Dass\\Desktop\\test\\Computer-Vision\\Social_distancing\\yolov3.weights"
#
detector = cv2.dnn.readNetFromCaffe(prototxt=protopath, caffeModel=modelpath)

# detector = cv2.dnn.readNetFromDarknet(modelConfiguration, modelWeight)
# detector.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)

#mention du nombre de classes ici
CLASSES = ["background", "aeroplane", "bicycle", "bird", "boat",
           "bottle", "bus", "car", "cat", "chair", "cow", "diningtable",
           "dog", "horse", "motorbike", "person", "pottedplant", "sheep",
           "sofa", "train", "tvmonitor"]

tracker = CentroidTracker(maxDisappeared=40, maxDistance=50)

In [10]:
def non_max_suppression_fast(boxes, overlapThresh):
    try:
        if len(boxes) == 0:
            return []

        if boxes.dtype.kind == "i":
            boxes = boxes.astype("float")

        pick = []

        x1 = boxes[:, 0]
        y1 = boxes[:, 1]
        x2 = boxes[:, 2]
        y2 = boxes[:, 3]

        area = (x2 - x1 + 1) * (y2 - y1 + 1)
        idxs = np.argsort(y2)

        while len(idxs) > 0:
            last = len(idxs) - 1
            i = idxs[last]
            pick.append(i)

            xx1 = np.maximum(x1[i], x1[idxs[:last]])
            yy1 = np.maximum(y1[i], y1[idxs[:last]])
            xx2 = np.minimum(x2[i], x2[idxs[:last]])
            yy2 = np.minimum(y2[i], y2[idxs[:last]])

            w = np.maximum(0, xx2 - xx1 + 1)
            h = np.maximum(0, yy2 - yy1 + 1)

            overlap = (w * h) / area[idxs[:last]]

            idxs = np.delete(idxs, np.concatenate(([last],
                                                   np.where(overlap > overlapThresh)[0])))

        return boxes[pick].astype("int")
    except Exception as e:
        print("Exception occurred in non_max_suppression : {}".format(e))
#Passez le lien vidéo ici

In [12]:
def main():
    #ouverture de la video avec openCv

    cap = cv2.VideoCapture('vid_short.mp4')

    #cap = cv2.VideoCapture(0)

    #prendre l'heure actuelle d'images par seconde avant l'execution
    fps_start_time = datetime.datetime.now()
    # fps = images par seconde // initié à 0
    fps = 0
    # nombre total d'images
    total_frames = 0
    # boucle de traitement de la video
    while True:
        # ici on la video image par image
        ret, frame = cap.read()
        # on redimensionne l'image d'une largeur de 600 px
        frame = imutils.resize(frame, width=600)
        # on increment total_frames de 1
        total_frames = total_frames + 1

        (H, W) = frame.shape[:2]

        blob = cv2.dnn.blobFromImage(frame, 0.007843, (W, H), 127.5)

        detector.setInput(blob)
        person_detections = detector.forward()
        rects = []
        for i in np.arange(0, person_detections.shape[2]):
            confidence = person_detections[0, 0, i, 2]
            if confidence > 0.5:
                idx = int(person_detections[0, 0, i, 1])

                if CLASSES[idx] != "person":
                    continue

                person_box = person_detections[0, 0, i, 3:7] * np.array([W, H, W, H])
                (startX, startY, endX, endY) = person_box.astype("int")
                rects.append(person_box)

        boundingboxes = np.array(rects)
        boundingboxes = boundingboxes.astype(int)
        rects = non_max_suppression_fast(boundingboxes, 0.3)
        centroid_dict = dict()
        objects = tracker.update(rects)
        for (objectId, bbox) in objects.items():
            x1, y1, x2, y2 = bbox
            x1 = int(x1)
            y1 = int(y1)
            x2 = int(x2)
            y2 = int(y2)
            cX = int((x1 + x2) / 2.0)
            cY = int((y1 + y2) / 2.0)


            centroid_dict[objectId] = (cX, cY, x1, y1, x2, y2)

            # text = "ID: {}".format(objectId)
            # cv2.putText(frame, text, (x1, y1-5), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (0, 0, 255), 1)
            
        red_zone_list = []
        for (id1, p1), (id2, p2) in combinations(centroid_dict.items(), 2):
            dx, dy = p1[0] - p2[0], p1[1] - p2[1]
            distance = math.sqrt(dx * dx + dy * dy)
            if distance < 75.0:
                if id1 not in red_zone_list:
                    red_zone_list.append(id1)
                if id2 not in red_zone_list:
                    red_zone_list.append(id2)

        for id, box in centroid_dict.items():
            if id in red_zone_list:
                cv2.rectangle(frame, (box[2], box[3]), (box[4], box[5]), (0, 0, 255), 2)
            else:
                cv2.rectangle(frame, (box[2], box[3]), (box[4], box[5]), (0, 255, 0), 2)


        fps_end_time = datetime.datetime.now()
        time_diff = fps_end_time - fps_start_time
        if time_diff.seconds == 0:
            fps = 0.0
        else:
            fps = (total_frames / time_diff.seconds)

        fps_text = "FPS: {:.2f}".format(fps)

        cv2.putText(frame, fps_text, (5, 30), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (0, 0, 255), 1)

        cv2.imshow("Social_Distancing", frame)
        key = cv2.waitKey(1)
        if key == ord('q'):
            break

    cv2.destroyAllWindows()

In [13]:
from scipy.spatial import distance as dist
# SciPy est un projet visant à unifier et fédérer un ensemble de bibliothèques Python à usage scientifique.
# Scipy utilise les tableaux et matrices du module NumPy donc il depend de NumPy
# Scipy.spatial peut calculer des triangulations, des diagrammes de Voronoi et
# des coques convexes d'un ensemble de points, en utilisant la bibliothèque Qhull .
# De plus, il contient des KDTree implémentations pour les requêtes de points du voisin le plus proche
# et des utilitaires pour les calculs de distance dans diverses métriques.
# distance permet le calcul de la matrice de distance à partir d'une collection de vecteurs d'observation
# bruts stockés dans un tableau rectangulaire.
from collections import OrderedDict
# OrderedDict sont des dictionnaires qui possèdent des capacités supplémentaires pour s'ordonner
#ils ont ont été conçus pour être performants dans les opérations de ré-arrangement.
# L'occupation mémoire, la vitesse de parcours et les performances de mise à jour étaient secondaires
# import numpy as np
# NumPy est une extension du langage de programmation Python,
# destinée à manipuler des matrices ou tableaux multidimensionnels ainsi que
# des fonctions mathématiques opérant sur ces tableaux

In [14]:
class CentroidTracker:
    def __init__(self, maxDisappeared=50, maxDistance=50):
        # initialise l'ID d'objet unique suivant avec deux
        # dictionnaires utilisés pour suivre le mappage d'un objet donné
        # ID à son point central et nombre d'images consécutives qu'il a
        # a été marqué comme "disparu", respectivement
        self.nextObjectID = 0
        self.objects = OrderedDict()
        self.disappeared = OrderedDict()
        self.bbox = OrderedDict()  # CHANGE

        # stocke le nombre d'images consécutives maximum pour une donnée
        # objet peut être marqué comme "disparu" jusqu'à ce que nous
        # aurons plus besoin de l'enregistrer dans la suite du traitement
        self.maxDisappeared = maxDisappeared

        # stocker la distance maximale entre les points centrals  à associer
        # un objet - si la distance est supérieure à ce maximum
        # distance, nous commencerons à marquer l'objet comme "disparu"
        self.maxDistance = maxDistance

    def register(self, centroid, inputRect):
        # lors de l'enregistrement d'un objet, nous utilisons le prochain objet disponible
        # ID pour stocker le point central
        self.objects[self.nextObjectID] = centroid
        self.bbox[self.nextObjectID] = inputRect  # CHANGE
        self.disappeared[self.nextObjectID] = 0
        self.nextObjectID += 1

    def deregister(self, objectID):
        # pour désenregistrer un ID d'objet, nous supprimons l'ID d'objet de
        # nos deux dictionnaires respectifs
        del self.objects[objectID]
        del self.disappeared[objectID]
        del self.bbox[objectID]  # CHANGE

    def update(self, rects):
        # vérifier pour voir si la liste des rectangles de boîte englobante d'entrée
        # est vide
        if len(rects) == 0:
            # boucle sur tous les objets suivis existants et marque-les
            # comme disparu
            for objectID in list(self.disappeared.keys()):
                self.disappeared[objectID] += 1

                # si nous avons atteint un nombre maximum de
                # cadres où un objet donné a été marqué comme
                # manquant, désenregistrer(deregister)
                if self.disappeared[objectID] > self.maxDisappeared:
                    self.deregister(objectID)

            # revenir tôt car il n'y a pas de point central ou d'informations de suivi
            # mettre à jour
            # return self.objects
            return self.bbox

        # initialise un tableau de centroïdes(points centrals) d'entrée pour l'image courante
        inputCentroids = np.zeros((len(rects), 2), dtype="int")
        inputRects = []
        # boucle sur les rectangles de la boîte englobante
        for (i, (startX, startY, endX, endY)) in enumerate(rects):
            # utiliser les coordonnées de la boîte englobante pour dériver le point central
            cX = int((startX + endX) / 2.0)
            cY = int((startY + endY) / 2.0)
            inputCentroids[i] = (cX, cY)
            inputRects.append(rects[i])  # CHANGE

        # si nous ne suivons actuellement aucun objet, prenez l'entrée
        # centroids(point centraux) et enregistrez chacun d'eux
        if len(self.objects) == 0:
            for i in range(0, len(inputCentroids)):
                self.register(inputCentroids[i], inputRects[i])  # CHANGE

        # sinon, des objets sont qui sont en suivi, alors nous devons donc
        # essayez de faire correspondre les point centraux d'entrée à l'objet existant
        # centroids
        else:
            # récupérer l'ensemble des ID d'objet et des centres de gravité correspondants
            objectIDs = list(self.objects.keys())
            objectCentroids = list(self.objects.values())

            # calculer la distance entre chaque paire d'objets
            # centroids et centroids d'entrée, respectivement - notre
            # objectif sera de faire correspondre un point central d'entrée à un
            # centre de l'objet
            D = dist.cdist(np.array(objectCentroids), inputCentroids)

            # pour effectuer cette correspondance, nous devons (1) trouver la
            # plus petite valeur de chaque ligne, puis (2) trier la ligne
            # index en fonction de leurs valeurs minimales afin que la ligne
            # avec la plus petite valeur seront stockées en tête de la
            # liste
            rows = D.min(axis=1).argsort()

            # ensuite, nous effectuons un processus similaire sur les colonnes en
            # trouver la plus petite valeur dans chaque colonne puis
            # trier à l'aide de la liste d'index des lignes précédemment calculée
            cols = D.argmin(axis=1)[rows]

            # afin de déterminer si nous devons mettre à jour, enregistrer(register function),
            # ou désenregistrer(deregister function) un objet dont nous devons garder la trace
            # des lignes et des index de colonnes que nous avons déjà examinés
            usedRows = set()
            usedCols = set()

            # boucle sur la combinaison de l'index (ligne, colonne)
            # tuples
            for (row, col) in zip(rows, cols):
                # si nous avons déjà examiné la ligne ou
                # valeur de colonne avant, on l'ignore
                if row in usedRows or col in usedCols:
                    continue

                # si la distance entre les points centraux est supérieure à
                # la distance maximale, n'associez pas les deux
                # centroids sur le même objet
                if D[row, col] > self.maxDistance:
                    continue

                # sinon, récupérez l'ID objet de la ligne courante,
                # définit son nouveau point central et réinitialise le
                # compteur
                objectID = objectIDs[row]
                self.objects[objectID] = inputCentroids[col]
                self.bbox[objectID] = inputRects[col]  # CHANGE
                self.disappeared[objectID] = 0

                # indique que nous avons examiné chacune des lignes et
                # index de colonnes, respectivement
                usedRows.add(row)
                usedCols.add(col)

            # calculer à la fois l'index de ligne et de colonne que nous n'avons PAS encore
            # examiné
            unusedRows = set(range(0, D.shape[0])).difference(usedRows)
            unusedCols = set(range(0, D.shape[1])).difference(usedCols)

            # dans le cas où le nombre de points central d'objet est
            # égal ou supérieur au nombre du point central d'entrée
            # nous devons vérifier et voir si certains de ces objets ont
            # potentiellement disparu
            if D.shape[0] >= D.shape[1]:
                # boucle sur les index de ligne inutilisés
                for row in unusedRows:
                    # récupérer l'ID d'objet de la ligne correspondante
                    # indexer et incrémenter le compteur disparu
                    objectID = objectIDs[row]
                    self.disappeared[objectID] += 1

                    # vérifier si le nombre de
                    # frames l'objet a été marqué "disparu"
                    # pour les mandats de désenregistrement de l'objet
                    if self.disappeared[objectID] > self.maxDisappeared:
                        self.deregister(objectID)

            # sinon, si le nombre des point centraux d'entrée est plus grand
            # que le nombre des points centraux d'objets existants dont nous avons besoin
            # enregistrer(register function) chaque nouveau point central d'entrée comme un objet traçable
            else:
                for col in unusedCols:
                    self.register(inputCentroids[col], inputRects[col])

        # renvoie l'ensemble des objets traçables
        # return self.objects
        return self.bbox

In [15]:
main()

AttributeError: 'NoneType' object has no attribute 'shape'

In [16]:
main()

AttributeError: 'NoneType' object has no attribute 'shape'